In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from torch.utils.data import DataLoader
import random
from model import BERTModel,Trainer
import torch.nn as nn
import torch.optim as optim
from datasets import BertDataset,ML20MDataset

In [2]:
# load dataset
mldataset = ML20MDataset("data/",4,5,5)

In [3]:
dataset = mldataset.load_dataset()

In [4]:
my_dataset = BertDataset(mldataset,'data/',128)

Negatives samples exist. Loading.


In [5]:
train,val,test = my_dataset.get_datasets()

In [6]:
# training settings
device = 'mps'
lr = 1e-3
epochs = 110
decay_steps=15
gamma = 0.01
weight_decay = 0.01
model = BERTModel(128,my_dataset.item_count,2,4,256,0.1)
optimizer = optim.Adam(model.parameters(),lr = lr, weight_decay= weight_decay)
loss_fn = nn.CrossEntropyLoss(ignore_index=0)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=decay_steps, gamma=gamma)
train_loader = DataLoader(train,batch_size=64,shuffle=True, drop_last=True)
val_loader = DataLoader(val,batch_size=64,shuffle=False, drop_last=True,pin_memory=True)

In [7]:
bert_trainer = Trainer(model,train_loader,val_loader,'./checkpoint/',device,optimizer,loss_fn,lr_scheduler,epochs)

In [8]:
# start training
bert_trainer.train()

  0%|          | 0/2135 [00:00<?, ?it/s]/Users/felix/Documents/datamining/RecModel/model/utils/utils.py:42: UserWarning: MPS: no support for int64 reduction ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:144.)
  answer_count = labels.sum(1)
Val: N@1 0.009, N@5 0.028, N@10 0.044, R@1 0.009, R@5 0.049, R@10 0.097: 100%|██████████| 2135/2135 [05:16<00:00,  6.75it/s]
Epoch 1, loss 8.679361 :   3%|▎         | 62/2135 [00:21<12:02,  2.87it/s] 


KeyboardInterrupt: 